In [2]:
import pandas as pd
from tqdm import tqdm

In [96]:
folder = './input_files/'

file = 'sentara-norfolk-general-hospital_standardcharges.xlsx'

var_name = 'payer_name'
value_name = 'standard_charge'

In [97]:
xls = pd.ExcelFile(folder + file)

In [98]:
sheets = xls.sheet_names
sheets.remove('Tab Summary')
sheets.remove('Discounted Cash Price')
sheets

['Gross Charges',
 'Pharmacy & Supply Charges',
 'Inpatient Minimum',
 'Inpatient Maximum',
 'IP Payer Specific Charge 1',
 'IP Payer Specific Charge 2',
 'IP Payer Specific Charge 3',
 'IP Payer Specific Charge 4',
 'IP Payer Specific Charge 5',
 'IP Payer Specific Charge 6',
 'IP Payer Specific Charge 7',
 'IP Payer Specific Charge 8',
 'IP Payer Specific Charge 9',
 'IP Payer Specific Charge 10',
 'IP Payer Specific Charge 11',
 'IP Payer Specific Charge 12',
 'IP Payer Specific Charge 13',
 'IP Payer Specific Charge 14',
 'IP Payer Specific Charge 15',
 'IP Payer Specific Charge 16',
 'IP Payer Specific Charge 17',
 'IP Payer Specific Charge 18',
 'IP Payer Specific Charge 19',
 'IP Payer Specific Charge 20',
 'IP Payer Specific Charge 21',
 'IP Payer Specific Charge 22',
 'IP Payer Specific Charge 23',
 'IP Payer Specific Charge 24',
 'IP Payer Specific Charge 25',
 'IP Payer Specific Charge 26',
 'IP Payer Specific Charge 27',
 'IP Payer Specific Charge 28',
 'IP Payer Specific C

gc = gross charges
psc = pharmacy & supply
dcp = discounted cash price


# Gross Charges

In [106]:
gc_df = pd.read_excel(xls, 'Gross Charges', dtype=str, skiprows=4)

In [107]:
gc_df.rename(columns={
    'CDM': 'local_code',
    'CDM Description': 'description',
    'Revenue Code': 'rev_code',
    'CPT/HCPCS': 'hcpcs_cpt',
    'Fee Schedule Name': 'additional_payer_specific_notes'
}, inplace=True)

In [111]:
cols = gc_df.columns.tolist()
id_vars = cols[:4]
value_vars = cols[4:]
if 'additional_payer_specific_notes' in cols:
    id_vars.append('additional_payer_specific_notes')
    value_vars.remove('additional_payer_specific_notes')

gc_df = pd.melt(gc_df, id_vars=id_vars, value_vars=value_vars, var_name=var_name, value_name=value_name)

In [113]:
if 'additional_payer_specific_notes' in cols:
    gc_df.loc[gc_df['payer_name'] != 'Fee Schedule', 'additional_payer_specific_notes'] = pd.NA

In [9]:
payer_mapping = {
    'Unit Price': 'gross',
    'Discount Cash Price': 'cash',
    'Fee Schedule': 'negotiated'
}

gc_df['rate_category'] = gc_df['payer_name'].map(payer_mapping)

In [10]:
sheets.remove('Gross Charges')

# Pharmacy & Supply

In [116]:
psc = pd.read_excel(xls, 'Pharmacy & Supply Charges', dtype=str, skiprows=4)

,CDM,CDM DESCRIPTION,HCPCS,Revenue Code,Unit Price,Discount Cash Price
0,50257585,"ABALOPARATIDE 80 MCG (3,120 MCG/1.56 ML) PEN I...",NaN,250,7334,3667
1,50257585,ACETAMINOPHEN 160 MG/5 ML (5 ML) SOLN,NaN,250,1,0.5
2,50257585,ACETAMINOPHEN 160MG/5ML SOLN,NaN,250,2,1
3,50257585,ACETAMINOPHEN 650 MG/20.3 ML SUSPENSION,NaN,250,1,0.5
4,50257585,ACETYCHOLINE CHLORIDE 1 % (10 MG/ML) KIT,NaN,250,147,73.5
...,...,...,...,...,...,...
32101,74597017,HUMAN PROTHROMBIN COMPLEX (KCENTRA/PCC4) 500 U...,J7168,636,8,4
32102,74597017,"RHO(D) IMMUNE GLOBULIN-MALTOSE 1,500 UNIT (300...",J2792,636,54,27
32103,74597017,RHO(D) IMMUNE GLOBULIN-MALTOSE 15000 UNIT(3000...,J2792,636,54,27
32104,74597017,"VON WILLEBRAND FACTOR (RECOMB) 1,300 (+/-) UNI...",J7179,636,10,5


In [12]:
psc.rename(columns={
    'CDM': 'local_code',
    'CDM DESCRIPTION': 'description',
    'HCPCS': 'hcpcs_cpt',
    'Revenue Code': 'rev_code'
}, inplace=True)

In [13]:
cols = psc.columns.tolist()
id_vars = cols[:4]
value_vars = cols[4:]

psc = pd.melt(psc, id_vars=id_vars, value_vars=value_vars, var_name=var_name, value_name=value_name)

In [14]:
psc['rate_category'] = psc['payer_name'].map(payer_mapping)

In [15]:
sheets.remove('Pharmacy & Supply Charges')

# Inpatient Minimum

In [117]:
ipmin = pd.read_excel(xls, 'Inpatient Minimum', dtype=str, skiprows=4)

,MS-DRG,Description,De-Identified Minimum Negotiated Charge
0,001,Heart Transplant Or Implant Of Heart Assist Sy...,143322
1,003,Ecmo Or Tracheostomy With Mv >96 Hours Or Prin...,134794
2,004,Tracheostomy With Mv >96 Hours Or Principal Di...,33372
3,010,Pancreas Transplant,31142
4,011,"Tracheostomy For Face, Mouth And Neck Diagnose...",26856
...,...,...,...
565,982,Extensive O.R. Procedures Unrelated To Princip...,5424
566,983,Extensive O.R. Procedures Unrelated To Princip...,5541
567,987,Non-Extensive O.R. Procedures Unrelated To Pri...,13791
568,988,Non-Extensive O.R. Procedures Unrelated To Pri...,7432


In [17]:
ipmin.rename(columns={
    'MS-DRG': 'ms_drg',
    'Description': 'description',
}, inplace=True)

In [18]:
cols = ipmin.columns.tolist()
id_vars = cols[:2]
value_vars = cols[2:]

ipmin = pd.melt(ipmin, id_vars=id_vars, value_vars=value_vars, var_name=var_name, value_name=value_name)

In [19]:
ipmin['rate_category'] = 'min'
ipmin['setting'] = 'inpatient'

In [20]:
sheets.remove('Inpatient Minimum')

# Inpatient Maximum

In [21]:
ipmax = pd.read_excel(xls, 'Inpatient Maximum', dtype=str, skiprows=4)

In [22]:
ipmax.rename(columns={
    'MS-DRG': 'ms_drg',
    'Description': 'description',
}, inplace=True)

In [23]:
cols = ipmax.columns.tolist()
id_vars = cols[:2]
value_vars = cols[2:]

ipmax = pd.melt(ipmax, id_vars=id_vars, value_vars=value_vars, var_name=var_name, value_name=value_name)

In [24]:
ipmax['rate_category'] = 'max'
ipmax['setting'] = 'inpatient'

In [25]:
sheets.remove('Inpatient Maximum')

# Outpatient Minimum

In [26]:
opmin = pd.read_excel(xls, 'Outpatient Minimum', dtype=str, skiprows=4)

opmin.rename(columns={
    'APC': 'apc',
    'Description': 'description',
}, inplace=True)

cols = opmin.columns.tolist()
id_vars = cols[:2]
value_vars = cols[2:]

opmin = pd.melt(opmin, id_vars=id_vars, value_vars=value_vars, var_name=var_name, value_name=value_name)

opmin['rate_category'] = 'min'
opmin['setting'] = 'outpatient'

mask = opmin['apc'].str.contains('N')
opmin.loc[mask, 'code'] = opmin['apc']
opmin.loc[mask, 'apc'] = pd.NA

sheets.remove('Outpatient Minimum')

# Outpatient Maximum

In [27]:
opmax = pd.read_excel(xls, 'Outpatient Maximum', dtype=str, skiprows=4)

opmax.rename(columns={
    'APC': 'apc',
    'Description': 'description',
}, inplace=True)

cols = opmax.columns.tolist()
id_vars = cols[:2]
value_vars = cols[2:]

opmax = pd.melt(opmax, id_vars=id_vars, value_vars=value_vars, var_name=var_name, value_name=value_name)

opmax['rate_category'] = 'max'
opmax['setting'] = 'outpatient'

mask = opmax['apc'].str.contains('N')
opmax.loc[mask, 'code'] = opmax['apc']
opmax.loc[mask, 'apc'] = pd.NA


sheets.remove('Outpatient Maximum')

# Inpatient Payers

In [28]:
ip_sheets = [x for x in sheets if x.startswith('IP')]

ip_dfs = []

In [29]:
for sheet in ip_sheets:
    df = pd.read_excel(xls, sheet, dtype=str)
    
    payer = df.iloc[2, 1]
    df.columns = df.loc[5]
    df = df.loc[6:]
    
    df.rename(columns={
    'MS-DRG': 'ms_drg',
    'Description': 'description',
    'Payer Specific Negotiated Charge': 'standard_charge'
    }, inplace=True)

    df['payer_name'] = payer
    df['rate_category'] = 'negotiated'

    ip_dfs.append(df)


In [30]:
ip_df = pd.concat(ip_dfs, ignore_index=True)

ip_df['setting'] = 'inpatient'

# Outpatient Payers

In [31]:
op_sheets = [x for x in sheets if x.startswith('OP')]

op_dfs = []

In [32]:
for sheet in op_sheets:
    df = pd.read_excel(xls, sheet, dtype=str)
    
    payer = df.iloc[2, 1]
    df.columns = df.loc[5]
    df = df.loc[6:]

    df.rename(columns={
        'APC': 'apc',
        'Description': 'description',
        'Payer Specific Negotiated Charge': 'standard_charge'
    }, inplace=True)

    df['payer_name'] = payer
    df['rate_category'] = 'negotiated'

    mask = df['apc'].str.startswith('N')
    df.loc[mask, 'code'] = df['apc']
    df.loc[mask, 'apc'] = pd.NA

    op_dfs.append(df)


In [33]:
op_df = pd.concat(op_dfs, ignore_index=True)

op_df['setting'] = 'outpatient'

In [34]:
dfs = [gc_df, psc, ipmin, ipmax, opmin, opmax, ip_df, op_df]

df = pd.concat(dfs)

In [95]:
df.drop_duplicates(subset='rate_category')

,local_code,description,rev_code,hcpcs_cpt,payer_name,standard_charge,rate_category,ms_drg,setting,apc,code
0,10400001,NEW PATIENT - EYE EXAM,0510,92002,Unit Price,409,gross,NaN,NaN,NaN,NaN
8065,10400001,NEW PATIENT - EYE EXAM,0510,92002,Discount Cash Price,0,cash,NaN,NaN,NaN,NaN
0,NaN,Ecmo Or Tracheostomy With Mv >96 Hours Or Prin...,NaN,NaN,De-Identified Minimum Negotiated Charge,191459,min,003,inpatient,NaN,NaN
0,NaN,Ecmo Or Tracheostomy With Mv >96 Hours Or Prin...,NaN,NaN,De-Identified Maximum Negotiated Charge,191459,max,003,inpatient,NaN,NaN
0,NaN,Ecmo Or Tracheostomy With Mv >96 Hours Or Prin...,NaN,NaN,Medicaid Managed Care - Optima,191459,negotiated,003,inpatient,NaN,NaN
